#### Attendee registration

In [9]:
import json
from ipynb.fs.full.gpt import OpenAI
attendees = json.load(open('mock-attendees.json'))
event = json.load(open('mock-event.json'))

In [29]:
openai = OpenAI(relevant_memory=[], full_message_history=[])


def generate_category_name(text):
    (_,
     _,
     _,
     current_context) = openai.generate_chat_context("""
        You are a help AI assistant for a conference. You are given a list of attendees
        and their interests. You need to create a category for each attendee based on
        their interests.
    """)
    current_context.append(
        openai.create_chat_message(role='user', content=text))
    return openai.create_chat_completion(current_context, 'gpt-4')


def clustering_by_interests(attendees):
    categories = []
    for attendee in attendees:
        interests = attendee['interests']
        category_name_prompt = f"""
            Create a single category name for a person with the following interests:
                {", ".join(interests)}
            If the category fits one of the categories below, use that instead:
                {", ".join(categories)}
            Only reply with the name of category. nothing else.
        """
        category_name = generate_category_name(category_name_prompt)
        categories.append(category_name)
        attendee['category'] = category_name
    return attendees


def generate_questions(event, attendees):
    questions = []
    processed = []
    current_attendee = attendees[0]
    for attendee in attendees:
        if attendee != current_attendee:
            current_attendee = attendee
        for attendee in attendees:
            pair = [current_attendee['full_name'], attendee['full_name']]
            not_processed = len([pair for pair in processed if current_attendee['full_name']
                                in pair and attendee['full_name'] in pair]) == 0
            print(pair, not_processed)
            if attendee != current_attendee and not_processed:
                question_prompt = f"""
                    Generate a list of engaging and quirky ice breaker questions
                    between {current_attendee}  and {attendee} and the event their attending.
                """
                (_,
                 _,
                 _,
                 current_context) = openai.generate_chat_context(f"""
                    You are a help AI assistant for a conference. You are given a list of attendees
                    and their interests. You need to generate a list of engaging and quirky ice breaker questions based on
                    their interests and related to the event their attending. event: {event}.
                """)
                current_context.append(openai.create_chat_message(
                    role='user', content=question_prompt))
                response = openai.create_chat_completion(
                    current_context, 'gpt-4')
                questions.append({
                    'attendees': [current_attendee['full_name'], attendee['full_name']],
                    'questions': response
                })
                processed.append(pair)
                print(questions)
    return questions


In [4]:
attendee_categories =  clustering_by_interests(attendees)
json.dump(attendee_categories, open('attendee-categories.json', 'w'), indent=2)

In [5]:
import pandas as pd

df = pd.DataFrame(attendee_categories)
by_category = df.sort_values(by='category')
category = ""
for _,row in by_category.iterrows():
    if category != row['category']:
        category = row['category']
        print("---")
    print(row['full_name'], "-", row['category'])


---
Avery Anderson - Data Science and AI
Chloe Adams - Data Science and AI
Lily Clark - Data Science and AI
Amelia Taylor - Data Science and AI
---
Leah Roberts - Emerging Technologies and AI
---
Zoe Thomas - Software and IoT Development
Scarlett White - Software and IoT Development
Emily Hall - Software and IoT Development
Natalie Jackson - Software and IoT Development
Ella Lewis - Software and IoT Development
Mia Wilson - Software and IoT Development
Isabella Davis - Software and IoT Development
Ava Williams - Software and IoT Development
Olivia Martin - Software and IoT Development
Harper Harris - Software and IoT Development
---
Charlotte Thompson - Technology and Security
Sophie Walker - Technology and Security
Grace Turner - Technology and Security
Sophia Brown - Technology and Security
Emma Johnson - Technology and Security
John Doe - Technology and Security


In [30]:
questions = generate_questions(event, attendees)
json.dump(questions, open('questions.json', 'w'), indent=2)

['John Doe', 'John Doe'] True
['John Doe', 'Emma Johnson'] True
[{'attendees': ['John Doe', 'Emma Johnson'], 'questions': "1. If you could create a new AI-powered IoT device to solve a common problem, what would it be and why?\n\n2. How do you think the combination of AI and cybersecurity will shape the future of online privacy?\n\n3. In your opinion, what is the most exciting development in AI and Big Data analytics in recent years?\n\n4. If you could collaborate on a project that combines your interests in AI, IoT, and Machine Learning, what would it be about?\n\n5. How do you think AI and Machine Learning can help improve cybersecurity measures in the Internet of Things?\n\n6. What was your favorite workshop or hands-on session at the Tech Innovators Conference 2023 so far, and why?\n\n7. If you could have a conversation with any AI pioneer, who would it be and what would you ask them?\n\n8. How do you think the advancements in AI and Big Data analytics will impact the way businesse

In [24]:
processed = [
    ['a', 'b'],
    ['b', 'a'],
    ['a', 'c'],
    ['c', 'a'],
]

[pair for pair in processed if 'z' in pair and 'b' in pair]


False